In [151]:
from abc import ABC, abstractmethod

In [35]:
class Board:
    def __init__(self):
        self.board = [[0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0]]
        
        self.blank = '  '
        self.winner = "none"
        self.captured_pieces_white = []
        self.captured_pieces_black = []
        self.history = []
    def __str__(self):
        disp_board = ""
        for i in range(64):
            
            if i%8 == 0:
                disp_board += str(8-int(i/8)) + " "
            
            this_space = self.board[int(i/8)][i%8]
            
            piece_to_add = str(this_space) + "|"
            disp_board += piece_to_add
            
            if i%8 == 7:
                disp_board = disp_board[:-1] + "\n"
        disp_board += "  a b c d e f g h"
        return disp_board

    def show_history(self, player_input):
        return self.history

    def split_notation(self, notation):
        notation_rank = list(notation)[1]
        notation_file = list(notation)[0]
        rank_to_y = {"a":0, "b":1, "c":2, "d":3, "e":4, "f":5, "g":6, "h":7}
        notation_x = 8 - int(notation_rank)
        notation_y = rank_to_y.get(notation_file)
        return notation_x, notation_y

    def get_piece_at(self, position):

        position_xy = self.split_notation(position)
        i = position_xy[0]
        j = position_xy[1]
        return self.board[i][j]

    # checking move validity

    def is_piece(self, position):

        if self.get_piece_at(position) == 0:
            return True
        else:
            return False

    # uppercase is white
    
    def is_white(self, position):
        
        return self.get_piece_at(position).isupper()
        
    def move_is_valid(self, origin, destination):

        if is_piece(origin):
            
            # get valid moves from Piece class

            if not is_piece(destination):
                return True

            elif is_white(origin) == is_white(destination):
                return False

            else:
                return True
            
        else:
            return("No piece there")

In [39]:
chess = Board()
chess.board[3][5] = "N"
chess.board[2][3] = "P"
chess.move(

8 0|0|0|0|0|0|0|0
7 0|0|0|0|0|0|0|0
6 0|0|0|P|0|0|0|0
5 0|0|0|0|0|N|0|0
4 0|0|0|0|0|0|0|0
3 0|0|0|0|0|0|0|0
2 0|0|0|0|0|0|0|0
1 0|0|0|0|0|0|0|0
  a b c d e f g h


In [40]:
chess.is_allied("f5", "d6")

True